<a href="https://colab.research.google.com/github/ainfanzon/Cockroach_IAM_Workshop/blob/main/GCP_Colab_notebooks/Exercise_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://drive.google.com/uc?id=1XYr9Tyrz31a5kZdo601xD1QWz_YM8-H3">

### CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.

# Identity and Access Management Workshop.
---
## Multiregional Support.

CockroachDB's distributed architecture allows it to replicate data across multiple nodes in different [geographic regions](https://www.cockroachlabs.com/docs/stable/multiregion-overview), ensuring that data is available even if one or more nodes or entire regions become unavailable.

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
          <img src="https://drive.google.com/uc?id=14Mm3_GcslyAkF2pHiD2tbsPsZLMLQQxP" width="800" height="345">
      </td>
  </tr>
</table>

</body>
</html>

In this section you will:

1. Learn multiregional concepts.
1. Launch a nine nodes cluster.
1. Create and load the **movr** database.
1. Convert it to a multiregional database.
1. Configure regional by row.
1. Setting Regional Survival Goal

---

## 1. Muliregional Concepts.

There are three basic concepts in a multiregional deployment, namely:

- Region - A geographical location that has a set of CockroachDB nodes deployed within it. One of the regions is defined as *primary* and it is the default for all the tables in the database.
- Zones - A region has one or more zones. Zones are typically aligned with data centers within a region.
- Survival goals - The survival goals in a multiregional deployment are:
  - Zone survival - Is the default setting. It ensures that the database remains available for reads and writes if a zone becomes unavailable.
  - Region survival - Ensures that the database remains available for reads and writes even if an entire region becomes unavailable. This comes at the cost of increased write latency, but read performance is unaffected.
  - Locality - distribution within a table to optimize access from specific regions
  - Global table - optimized fro low-latency reads from any region
  - Regional table - Optimized for low-latency reads and writes from a single region.
  - Regional by row - optimized for low-latency reads and writes for a region. Rows are assigned to specific regions.

---

## 2. Launch a nine nodes cluster.

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td align="right">
          <img src="https://drive.google.com/uc?id=1YQ55Zp8KLy_ae7REkHG3TybeHU2KUDwk" width="850" height="450">
      </td>
  </tr>
</table>

</body>
</html>
<br>

To start a cluster with nine nodes execute the steps below:

- On your laptop open a terminal window and connect to the GCP compute engine using ssh.
<br>
<html>
<body>
<table style="width:100%"; border="1">
  <tr>
      <th>AWS</th>
      <th>GCP</th>
  </tr>
  <tr>
      <td align="right">
          ssh -i "&lt;Key_Pair.pem&gt;" &lt;UserId&gt;@&lt;Public IP&gt;
      </td>
      <td align="right">
          ssh &lt;UserId&gt;@&lt;Public IP&gt;
      </td>
  </tr>
</table>
</body>
</html>
<br>

- To start a the cluster execute the `mr_strt_crdb.sh`located in the `/home/cockroach/scripts/` directory.
<br><br>
  NOTE: Run the `/home/cockroach/scripts/cleanup.sh` sript to remove any active nodes from the previous lab.
<br><br>
<code>
  /home/cockroach/scripts/mr_start_crdb.sh
</code>
<br><br>
  The script will execute the start command nine times
<br>
<p>cockroach start<br>
&emsp;&emsp;--insecure<br>
&emsp;&emsp;--listen-addr=&lt;ip address&gt;:&lt;sql listening port&gt;<br>
&emsp;&emsp;--join=&lt;ip address&gt;:&lt;sql listening port&gt;, ... ,&lt;ip address&gt;:&lt;sql listening port&gt;<br>
&emsp;&emsp;--http-addr=&lt;ip address&gt;:&lt;http listening port&gt;<br>
&emsp;&emsp;--locality=region=us-west,zone=us-west-1a<br> &emsp;&emsp;--store=/home/cockroach/data/cr_data_1<br>
&emsp;&emsp;--background<br>
<br>
cockroach init --insecure --host &lt;ip address&gt;
<br><br>
You should see a `Cluster successfully initialized` message.
</p>

- Verify the cluster deployment.

> <code>
pgrep -a cockroach | awk '{ print $5}'<br><br>
--listen-addr=10.0.1.2:26257<br>
--listen-addr=10.0.1.2:26258<br>
--listen-addr=10.0.1.2:26261<br>
--listen-addr=10.0.1.2:26262<br>
--listen-addr=10.0.1.2:26259<br>
--listen-addr=10.0.1.2:26260<br>
--listen-addr=10.0.1.2:26263<br>
--listen-addr=10.0.1.2:26264<br>
--listen-addr=10.0.1.2:26265<br>
</code>

- Open another browser tab to display the cockroach DB Console http://35.235.99.125:8080/#/overview/list.

---

## 3. Create and load the **iam** and **northwind** databases.

<br>
<html>
<body>
<table style="width:100%">
  <tr>
    <th></th>
    <th>IAM Database Schema</th>
    <th></th>
    <th>Northwind Database Schema</th>
  </tr>
  <tr>
    <td style="background-color:#909090;"> </td>
    <td align="center">
      <img  align="left" src="https://drive.google.com/uc?id=114TZWQzz3Zv265CxHaSu03dn3idtR0Nj" width="525" height="325">
    </td>
    <td style="background-color:#f7f9fc width:1px" ></td>
    <td align="center">
      <img src="https://drive.google.com/uc?id=1eM0otn7ieCvBMXVQ0WmXgRKUf41GaS9h" width="850"
          height="650">
    </td>
  </tr>
</table>
</body>
</html>

Follow the steps below to create and load the databases:

- On one terminal change to the **/home/cockroach/dump** directory and execute the **Python** http server.

> ```
cd /home/cockroach/dump
python -m http.server 3000
```
You should see the HTTP server is running on port 3000

> ```
Serving HTTP on 0.0.0.0 port 3000 (http://0.0.0.0:3000/) ...
```

- On the second terminal execute the **iam.sql** and **northwind.sql** scripts to create the schema and populate the database.

    - First update the script with your PRIVATE IP address
    <br><br>
```
sed -E -i s/HOST_IP/$(hostname -I | awk '{print $1}')/ /home/cockroach/sql/iam.sql
sed -E -i s/HOST_IP/$(hostname -I | awk '{print $1}')/ /home/cockroach/sql/northwind.sql
```
<br>
    - Execute the IAM SQL script
    <br><br>
<code>
cockroach sql --host $(hostname -I) -u root -d default -f /home/cockroach/sql/iam.sql --insecure
</code>
<br><br>

    - Execute the Northwind SQL script
    <br><br>
<code>
cockroach sql --host $(hostname -I) -u root -d default -f /home/cockroach/sql/northwind.sql --insecure
</code>



Connect to the database

In [52]:
import psycopg2
import pandas as pd
from pandas import Series

from IPython.display import IFrame, display, HTML, Markdown

pd.set_option('display.max_colwidth', None)

try:
    conn = psycopg2.connect(
        database = 'iam'
      , user = 'root'
      , host = '34.221.252.230'      # Use the PUBLIC IP
      , port = '26257'
      , sslmode = 'disable'
    )
    display(Markdown("## Connection successful!"))
   # cursor = conn.cursor()
except psycopg2.OperationalError as e:
    print(f"Error connecting to database: {e}")

## Connection successful!


<html>
<body>
  <div class="container">
    <div class="image">
      <img style=”float: right; padding: 0px 0px 0px 3px;” align="right" src="https://drive.google.com/uc?id=1DvrZ4ZJpxFgFxgAuAnB-RN0WapuT_54u" width="450" height="210">
    </div>
    <div style="text-align: center; margin-top: 5px; padding: 13px 0 10px 0;">
      <p>Execute the <code>SHOW REGIONS;</code> command to display:
      <br>
      <ul>
        <li>The name of the region.</li>
        <li>The availability zones for the region.</li>
        <li>A set of database names that use the region.</li>
        <li>A set of database names for which the region is the primary region.</li>
        <li>A set of database names for which the region is the secondary region.</li>
      </.ul>
    </div>
  </div>
</body>
</html>

In [53]:
try:
    cursor = conn.cursor()
    cursor.execute("""
        SHOW REGIONS;
    """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    df_result_set = df_result_set.reset_index(drop=True)
    display(df_result_set)
    cursor.close()
except psycopg2.OperationalError as e:
    cursor.close()
    conn.commit()

,region,zones,database_names,primary_region_of,secondary_region_of
0,europe,"[eu-west-1a, eu-west-1b, eu-west-1c]",[iam],[],[]
1,us-east,"[us-east-1a, us-east-1b, us-east-1c]",[iam],[],[]
2,us-west,"[us-west-1a, us-west-1b, us-west-1c]",[iam],[iam],[]


- In which region is the **iam** database located?
- How many _zones_ does each _region_ has?

<html>
<body>
  <div class="container">
    <div class="image">
      <img style=”float: right; padding: 0px 0px 0px 3px;” align="right" src="https://drive.google.com/uc?id=1Gejgp5qbI2b4udQRGytEIxS0OvGykiT5" width="450" height="210">
    </div>
    <div style="text-align: center; margin-top: 5px; padding: 13px 0 10px 0;">
      <p>Execute the <code>SHOW RANGES FROM DATABASE movr;</code> command to display:
      <br>
      <ul>
        <li>The start key for the range.</li>
        <li>The end key for the range.</li>
        <li>The nodes that contain the range's replicas.</li>
        <li>The localities of the range's replicas.</li>
        <li>	The nodes that contain the range's voting replicas (that is, the replicas that participate in Raft elections).</li>
      </.ul>
    </div>
  </div>
</body>
</html>

In [54]:
try:
    cursor = conn.cursor()
    cursor.execute("""
      SELECT range_id, replicas, lease_holder
      FROM [SHOW RANGES FROM TABLE iam_users WITH DETAILS]
    """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    display(df_result_set)
    cursor.close()
except psycopg2.OperationalError as e:
    cursor.close()
    conn.commit()

,range_id,replicas,lease_holder
0,171,"[1, 2, 3, 4, 7]",3
1,329,"[1, 2, 3, 4, 7]",1
2,331,"[2, 4, 7, 8, 9]",8
3,332,"[1, 3, 4, 7, 9]",1
4,333,"[1, 5, 6, 7, 9]",5
5,334,"[1, 3, 4, 7, 9]",1


- Which nodes contain replicas of the iam database?
- Where is the replica set leader located?

---

## 4. Convert it to a multiregion database.

The SHOW REGIONS statement lists the cluster regions for a multiregional cluster, or the database regions for the databases in a multi-region cluster.

Assign Primary Region and Associate Non-Primary Regions to the **iam** Database

<html>
<head>
</head>
<body>
<table>
  <tr>
      <td>
          <table>
              <tr>
                  <th>Region</th>
                  <th>Availability Zone</th>          
              </tr>
              <tr>
                  <td>us-east1</td>
                  <td>us-east1a<br>us-east1b<br>us-east1c</td>
              </tr>
              <tr>
                  <td>us-west1</td>
                  <td>us-west1b</td>
              </tr>
              <tr>
                  <td>europe-west1</td>
                  <td>europe-west1c</td>
              </tr>
          </table>
      </td>
      <td>
          <img align="left" src="https://drive.google.com/uc?id=18ZUG07gpni11RISjm6jv11sK7_k6DR2x" width="500" height="300">
      </td>
  </tr>
</table>
</body>
</html>
<br>

To convert the movr database to multiregion, you need to assign a primary region and associate the nonprimary regions with the database.

- A multi-region deployment requires an enterprise license.
<br>

> ```echo $(hostname -I) | xargs -I {} cockroach sql --insecure --host={}:26257 -d defaultdb -f /home/cockroach/sql/apply_lic.sql```



- Associate the database to a primary and secondary regions.
<br>

> ```
echo $(hostname -I) | xargs -I {} cockroach sql --insecure --host={}:26257 -u root -d iam --execute="ALTER DATABASE iam SET PRIMARY REGION='us-west';"
echo $(hostname -I) | xargs -I {} cockroach sql --insecure --host={}:26257 -u root -d iam --execute="ALTER DATABASE iam ADD REGION 'us-east';"
echo $(hostname -I) | xargs -I {} cockroach sql --insecure --host={}:26257 -u root -d iam --execute="ALTER DATABASE iam ADD REGION 'europe';"
```

In [55]:
try:
    cursor = conn.cursor()
    cursor.execute("""
        SELECT name, locality
        FROM crdb_internal."tables"
        WHERE schema_name='public'
            AND database_name = 'iam'
        """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    display(df_result_set)
    cursor.close()
except psycopg2.OperationalError as e:
    conn.commit()
    cursor.close()

,name,locality
0,audit_log,REGIONAL BY TABLE IN PRIMARY REGION
1,auth_provider,REGIONAL BY TABLE IN PRIMARY REGION
2,iam_locations,REGIONAL BY TABLE IN PRIMARY REGION
3,iam_roles,REGIONAL BY TABLE IN PRIMARY REGION
4,iam_users,REGIONAL BY ROW
5,permissions,REGIONAL BY TABLE IN PRIMARY REGION
6,role_permission,REGIONAL BY TABLE IN PRIMARY REGION


## 5. Configure Regional by row

<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
</style>
</head>
<body>

<table style="width:100%">
  <tr>
      <td>
          <img align="left" src="https://drive.google.com/uc?id=1NxDSdkeSOiBbpR7O40IdCufzPXE8Vs8d" width="500" height="300">
      </td>
      <td>
          <table style="width:100%">
              <tr>
                  <th>Region</th>
                  <th>Row Location</th>          
              </tr>
              <tr>
                  <td>us-east1</td>
                  <td>new york<br>boston<br>washington dc<br><br><em>others</em></td>
              </tr>
              <tr>
                  <td>us-west1</td>
                  <td>san francisco<br>seattle<br>los angeles</td>
              </tr>
              <tr>
                  <td>europe-west1</td>
                  <td>amsterdam<br>paris<br>rome</td>
              </tr>
          </table>
      </td>
  </tr>
</table>
    
Assign rows to regions based on a given value


In [ ]:
try:
    conn.commit()
    cursor = conn.cursor()
    cursor.execute("""
        ALTER TABLE iam_users ADD COLUMN crdb_region crdb_internal_region AS (
            CASE
                WHEN user_region = 'Western' THEN 'us-west'
                WHEN user_region = 'Eastern' THEN 'us-east'
                WHEN user_region = 'European' THEN 'europe'
                ELSE 'us-west'
            END
        ) STORED
        """)
    cursor.close()
    display(Markdown("## Alter table completed!"))
except psycopg2.OperationalError as e:
    conn.commit()
    cursor.close()

In [56]:
try:
    conn.commit()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT DISTINCT user_region AS "User Region", crdb_region AS "CRDB Region"
        FROM iam_users;
        """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    df_result_set.set_index('CRDB Region', inplace=True)
    display(df_result_set)
    cursor.close()
except psycopg2.OperationalError as e:
    conn.commit()
    cursor.close()

,User Region
CRDB Region,
us-west,us-west
us-east,Eastern
us-west,None
us-west,Western
europe,European


In [ ]:
try:
    conn.commit()
    cursor = conn.cursor()
    cursor.execute("""
        ALTER TABLE iam_users ALTER COLUMN crdb_region SET NOT NULL
        """)
    cursor.close()
    display(Markdown("## Alter table SET NOT NULL completed!"))
except psycopg2.OperationalError as e:
    conn.commit()
    cursor.close()

## Alter table completed!

In [ ]:
try:
    conn.commit()
    cursor = conn.cursor()
    cursor.execute("""
        ALTER TABLE iam_users SET LOCALITY REGIONAL BY ROW;
        """)
    cursor.close()
    display(Markdown("## Alter table SET LOCALITY REGIONAL BY ROW completed!"))
except psycopg2.OperationalError as e:
    conn.commit()
    cursor.close()

## Alter table SET LOCALITY REGIONAL BY ROW completed!

In [57]:
try:
    conn.commit()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT name AS "Table Name"
            , locality AS "Locality"
        FROM crdb_internal."tables"
        WHERE schema_name='public'
            AND database_name = 'iam'
        """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    df_result_set.set_index('Table Name', inplace=True)
    display(df_result_set)
    cursor.close()
except psycopg2.OperationalError as e:
    conn.commit()
    cursor.close()

,Locality
Table Name,
audit_log,REGIONAL BY TABLE IN PRIMARY REGION
auth_provider,REGIONAL BY TABLE IN PRIMARY REGION
iam_locations,REGIONAL BY TABLE IN PRIMARY REGION
iam_roles,REGIONAL BY TABLE IN PRIMARY REGION
iam_users,REGIONAL BY ROW
permissions,REGIONAL BY TABLE IN PRIMARY REGION
role_permission,REGIONAL BY TABLE IN PRIMARY REGION


In [58]:
try:
    conn.commit()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT job_type
             , description
             , status
        FROM [show jobs]
        WHERE description LIKE '%REGIONAL%'
    """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    display(df_result_set)
    cursor.close()
except psycopg2.OperationalError as e:
    conn.commit()
    cursor.close()

,job_type,description,status
0,SCHEMA CHANGE,ALTER TABLE iam.public.iam_users SET LOCALITY REGIONAL BY ROW,succeeded


In [59]:
try:
    conn.commit()
    cursor = conn.cursor()
    cursor.execute("""
        SELECT DISTINCT index_name AS "Index Name"
            , column_Name AS "Column Name"
        FROM crdb_internal.index_columns
        WHERE descriptor_name = 'iam_users'
    """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    display(df_result_set)
    cursor.close()
except psycopg2.OperationalError as e:
    conn.commit()
    cursor.close()

,Index Name,Column Name
0,pk_user_id,crdb_region
1,pk_user_id,user_id


In [67]:
try:
    conn.commit()
    cursor = conn.cursor()
    cursor.execute("""
        SHOW RANGE FROM INDEX iam_users@primary FOR ROW ('us-west',250)
    """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    display(df_result_set)
    cursor.close()
except psycopg2.OperationalError as e:
    conn.commit()
    cursor.close()

,start_key,end_key,range_id,lease_holder,lease_holder_locality,replicas,replica_localities,voting_replicas,non_voting_replicas
0,"…/""@""/PrefixEnd",<after:/Table/137/4>,334,1,"region=us-west,zone=us-west-1a","[1, 3, 4, 7, 9]","[region=us-west,zone=us-west-1a, region=us-west,zone=us-west-1b, region=us-west,zone=us-west-1c, region=us-east,zone=us-east-1c, region=europe,zone=eu-west-1b]","[1, 4, 3]","[7, 9]"


---
## CockroachDB is a distributed SQL database that is __*highly scalable*__, __*resilient*__, and __*easy to use*__.
<img src="https://drive.google.com/uc?id=1XYr9Tyrz31a5kZdo601xD1QWz_YM8-H3">

---

# Appendix

Workshop CRDB user id and passowrd

> <p>uid = roachie<br>
pwd = roachfan
</p>

List CRDB process id and process name.

> <code>pgrep -l cockroach</code>

List the listening address of each `cockroach` process.

> <code>pgrep -a cockroach | awk '{ print $5}'</code>

Kill ALL CRDB processes

> <code>kill -9  $(pgrep cockroach)</code>

Remove all CRDB files

> <code>sudo rm -fR /home/cockroach/data/*</code>

## Queries

### Reconnect to the IAM **database**

### List 5 users in the IAM database

In [ ]:
try:
    cursor.execute("""
      SELECT
          user_name
        , email
      FROM iam_users
      LIMIT 5
    """)
    result_set = cursor.fetchall()
    df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
    display(df_result_set.style.hide(axis="index"))
except psycopg2.OperationalError as e:
    cursor.close()
    conn.commit()

user_name,email
lgethin0,kcamber0@google.com.au
spress1,labramamov1@cbc.ca
jhenric2,dbasset2@china.com.cn
acommin3,jjerrolt3@so-net.ne.jp
cpfeiffer4,nmumm4@goo.ne.jp


### Find all roles assigned to a specific user

In [ ]:
cursor.execute("""
SELECT
    user_name AS "User Name"
  , role_name AS "Role"
FROM iam_roles INNER JOIN iam_users USING (role_id)
WHERE user_name = 'acommin3'
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

User Name,Role
acommin3,Auditor


### List all roles in the Keycloak database

In [ ]:
cursor.execute("""
SELECT role_name
FROM iam_roles
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

role_name
Admin/Super Admin
User Manager
Group Manager
Auditor
Helpdesk Support
Developer
External User


### Find all users with a specific role

In [ ]:
cursor.execute("""
SELECT user_name
FROM iam_users
WHERE user_id IN (
  SELECT user_id
  FROM iam_roles
  WHERE role_id = (
    SELECT role_id
    FROM iam_roles
    WHERE role_name = 'Admin/Super Admin'
  )
)
LIMIT 5
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

user_name
lgethin0
spress1
jhenric2
acommin3
cpfeiffer4


### Find all users assigned to a specific group
SELECT username FROM users WHERE id IN (SELECT user_id FROM user_groups WHERE group_id = (SELECT id FROM groups WHERE group_name = 'example_group'));

In [ ]:
cursor.execute("""
SELECT
    user_name AS "User Name"
  , log_timestamp AS "Log Entry Time"
  , activity AS "Activity Description"
FROM iam_users INNER join audit_log using (user_id)
WHERE log_timestamp < current_date() - interval '30 days'
LIMIT 10
""")
result_set = cursor.fetchall()
df_result_set = pd.DataFrame(result_set, columns=[desc[0] for desc in cursor.description])
df_result_set.style.hide(axis="index")

User Name,Log Entry Time,Activity Description
smaggoriniw,2024-07-16 12:05:58,Aliquam quis turpis eget elit sodales scelerisque. Mauris sit amet eros. Suspendisse accumsan tortor quis turpis. Sed ante. Vivamus tortor. Duis mattis egestas metus. Aenean fermentum. Donec ut mauris eget massa tempor convallis.
bkliner1x,2023-11-16 19:35:59,Mauris lacinia sapien quis libero. Nullam sit amet turpis elementum ligula vehicula consequat. Morbi a ipsum.
bvatcher3m,2024-02-06 23:42:00,"Proin leo odio, porttitor id, consequat in, consequat ut, nulla. Sed accumsan felis."
bbegin41,2024-03-09 21:03:31,"Aenean sit amet justo. Morbi ut odio. Cras mi pede, malesuada in, imperdiet et, commodo vulputate, justo. In blandit ultrices enim. Lorem ipsum dolor sit amet, consectetuer adipiscing elit. Proin interdum mauris non ligula pellentesque ultrices. Phasellus id sapien in sapien iaculis congue. Vivamus metus arcu, adipiscing molestie, hendrerit at, vulputate vitae, nisl."
mrosettini4y,2024-08-29 22:18:49,Cras non velit nec nisi vulputate nonummy.
trothchild6d,2024-09-10 03:23:15,"Cum sociis natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus."
mpigginsej,2023-11-23 22:06:52,"Nam ultrices, libero non mattis pulvinar, nulla pede ullamcorper augue, a suscipit nulla elit ac nulla. Sed vel enim sit amet nunc viverra dapibus. Nulla suscipit ligula in lacus. Curabitur at ipsum ac tellus semper interdum. Mauris ullamcorper purus sit amet nulla. Quisque arcu libero, rutrum ac, lobortis vel, dapibus at, diam. Nam tristique tortor eu pede."
ikuschkego,2024-04-12 08:48:08,"Nunc nisl. Duis bibendum, felis sed interdum venenatis, turpis enim blandit mi, in porttitor pede justo eu massa. Donec dapibus. Duis at velit eu est congue elementum. In hac habitasse platea dictumst. Morbi vestibulum, velit id pretium iaculis, diam erat fermentum justo, nec condimentum neque sapien placerat ante. Nulla justo. Aliquam quis turpis eget elit sodales scelerisque. Mauris sit amet eros. Suspendisse accumsan tortor quis turpis."
psandlandk6,2024-02-08 00:50:52,Proin risus.
bluppittrf,2024-06-28 00:00:57,Etiam faucibus cursus urna. Ut tellus. Nulla ut erat id mauris vulputate elementum. Nullam varius. Nulla facilisi. Cras non velit nec nisi vulputate nonummy. Maecenas tincidunt lacus at velit. Vivamus vel nulla eget eros elementum pellentesque.




-- Find all users assigned to a specific provider:
SELECT email, phone, provider_name
FROM iam_users INNER JOIN auth_provider USING(provider_id)
WHERE provider_type = 'Multi-factor Authentication provider';

-- Find all users assigned to a specific group: SELECT username FROM users WHERE id IN (SELECT user_id FROM user_groups WHERE group_id = (SELECT id FROM groups WHERE group_name = 'example_group'));

-- Count the number of users in Keycloak:
SELECT COUNT(*) FROM iam_users;

-- Find all users who have not logged in for the past 30 days
SELECT user_name, log_timestamp
FROM iam_users INNER join audit_log using (user_id)
WHERE log_timestamp < current_date() - interval '30 days';